# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [221]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Generate the mathematical model

In [226]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [227]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Food and Beverage Production Planning

**Sets and Indices:**

*  $T = \{1, 2, ..., 12\}$: Set of time periods.
*  $F = \{1, 2\}$: Set of food products.
*  $B = \{1, 2, 3\}$: Set of beverage products. 

**Parameters:**

*  $FC_{it}$: Fixed cost of producing product $i \in F \cup B$ in time period $t \in T$. (Read from "fixed_cost_production.csv")
*  $VCprod_{it}$: Variable cost of producing one unit of product $i \in F \cup B$ in time period $t \in T$. (Read from "variable_cost_production.csv")
*  $VCstore_{it}$: Variable cost of storing one unit of product $i \in F \cup B$ in time period $t \in T$. (Read from "variable_cost_storage.csv")
*  $D_{it}$: Demand for product $i \in F \cup B$ in time period $t \in T$. (Read from "demand.csv")
*  $R_{it}$: Revenue per unit of product $i \in F \cup B$ sold in time period $t \in T$. (Read from "revenue.csv")
*  $SC_i$: Storage capacity for product $i \in F \cup B$. 
    *  $SC_1 = 580$ (Food 1)
    *  $SC_2 = 687$ (Food 2)
    *  $SC_3 = 599$ (Beverage 1)
    *  $SC_4 = 788$ (Beverage 2)
    *  $SC_5 = 294$ (Beverage 3)
*  $PC_i$: Production capacity for product $i \in F \cup B$.
    *  $PC_1 = 1080$ (Food 1)
    *  $PC_2 = 908$ (Food 2)
    *  $PC_3 = 408$ (Beverage 1)
    *  $PC_4 = 1000$ (Beverage 2)
    *  $PC_5 = 403$ (Beverage 3)

**Decision Variables:**

*  $X_{it}$: Quantity of product $i \in F \cup B$ produced in time period $t \in T$.
*  $S_{it}$: Quantity of product $i \in F \cup B$ stored at the end of time period $t \in T$. 
*  $Y_{it}$: Quantity of product $i \in F \cup B$ sold in time period $t \in T$.

**Objective Function:**
Maximize the total profit over the 12 time periods.

$Maximize \sum_{t \in T} \sum_{i \in F \cup B} (R_{it}Y_{it} - VCprod_{it}X_{it} - VCstore_{it}S_{it} - FC_{it} \cdot \mathbb{1}(X_{it}>0))$

where $\mathbb{1}(X_{it}>0)$ is an indicator function that equals 1 if $X_{it}>0$ and 0 otherwise. 

**Constraints:**

1. **Production Capacity:** The quantity produced in each time period cannot exceed the production capacity.
    *  $X_{it} \leq PC_i, \forall i \in F \cup B, \forall t \in T$

2. **Storage Capacity:** The quantity stored at the end of each time period cannot exceed the storage capacity.
    *  $S_{it} \leq SC_i, \forall i \in F \cup B, \forall t \in T$

3. **Demand:** The quantity sold in each time period cannot exceed the demand.
    *  $Y_{it} \leq D_{it}, \forall i \in F \cup B, \forall t \in T$

4. **Inventory Balance:** The quantity stored at the end of each period equals the quantity stored at the end of the previous period plus the quantity produced minus the quantity sold.
    *  $S_{i1} = X_{i1} - Y_{i1}, \forall i \in F \cup B$
    *  $S_{it} = S_{i(t-1)} + X_{it} - Y_{it}, \forall i \in F \cup B, \forall t \in T, t > 1$

5. **Non-Negative Production:**  
    *  $X_{it} \geq 0, \forall i \in F \cup B, \forall t \in T$

6. **Non-Negative Storage:**  
    *  $S_{it} \geq 0, \forall i \in F \cup B, \forall t \in T$

7. **Non-Negative Sales:**  
    *  $Y_{it} \geq 0, \forall i \in F \cup B, \forall t \in T$

8. **Integer Production for Food:**
    *  $X_{it} \in \mathbb{Z}, \forall i \in F, \forall t \in T$ 


## 3. Generate the pyomo code

In [228]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [229]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo
import pandas as pd

# Sample Data 
# Assuming data is in .csv files, replace with actual file paths if needed
fixed_cost_production_data = {
    ('Food1', 1): 100, ('Food1', 2): 105, ('Food1', 3): 110, # ... and so on for all periods and products
    ('Food2', 1): 150, ('Food2', 2): 155, ('Food2', 3): 160, # ... 
    ('Beverage1', 1): 80, ('Beverage1', 2): 85, ('Beverage1', 3): 90, # ... 
    ('Beverage2', 1): 120, ('Beverage2', 2): 125, ('Beverage2', 3): 130, # ... 
    ('Beverage3', 1): 70, ('Beverage3', 2): 75, ('Beverage3', 3): 80, # ... 
}
variable_cost_production_data = {
    # similar structure as fixed_cost_production_data
}
variable_cost_storage_data = {
    # similar structure as fixed_cost_production_data
}
demand_data = {
    # similar structure as fixed_cost_production_data
}
revenue_data = {
    # similar structure as fixed_cost_production_data
}

# Create Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, 13))  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[1, 2, 3])  # Beverage products
model.FB = model.F | model.B # All Products

# Parameters
model.FC = pyo.Param(model.FB, model.T, initialize=fixed_cost_production_data) 
model.VCprod = pyo.Param(model.FB, model.T, initialize=variable_cost_production_data)
model.VCstore = pyo.Param(model.FB, model.T, initialize=variable_cost_storage_data)
model.D = pyo.Param(model.FB, model.T, initialize=demand_data)
model.R = pyo.Param(model.FB, model.T, initialize=revenue_data)

# Storage and Production Capacities - you already have this data
model.SC = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
model.PC = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Decision Variables
model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity produced
model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity sold
model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity stored

# Integer requirement for Food Production
model.X_integer = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)
def x_linking_rule(model, i, t):
    if i in model.F:
        return model.X[i,t] == model.X_integer[i,t]
    else:
        return pyo.Constraint.Skip
model.x_linking = pyo.Constraint(model.FB, model.T, rule=x_linking_rule) 

# Objective Function
def objective_rule(model):
    return sum(model.R[i, t] * model.Y[i, t] - model.VCprod[i, t] * model.X[i, t] - model.VCstore[i, t] * model.S[i, t] - model.FC[i, t] * (model.X[i, t] > 0) for i in model.FB for t in model.T)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints

# 1. Production Capacity
def production_capacity_rule(model, i, t):
    return model.X[i, t] <= model.PC[i]
model.production_capacity = pyo.Constraint(model.FB, model.T, rule=production_capacity_rule)

# 2. Storage Capacity
def storage_capacity_rule(model, i, t):
    return model.S[i, t] <= model.SC[i]
model.storage_capacity = pyo.Constraint(model.FB, model.T, rule=storage_capacity_rule)

# 3. Demand
def demand_rule(model, i, t):
    return model.Y[i, t] <= model.D[i, t]
model.demand = pyo.Constraint(model.FB, model.T, rule=demand_rule)

# 4. Inventory Balance
def inventory_balance_rule(model, i, t):
    if t == 1:
        return model.S[i, t] == model.X[i, t] - model.Y[i, t]
    else:
        return model.S[i, t] == model.S[i, t-1] + model.X[i, t] - model.Y[i, t]
model.inventory_balance = pyo.Constraint(model.FB, model.T, rule=inventory_balance_rule)

# Solve the model
solver = pyo.SolverFactory('cbc') # Choose a solver, 'cbc' is a free open-source solver
results = solver.solve(model)

# Print Results
print(results)

# Access the optimal values of decision variables
for i in model.FB:
    for t in model.T:
        print(f"Product {i}, Time {t}: Production = {pyo.value(model.X[i,t])}, Storage = {pyo.value(model.S[i,t])}, Sales = {pyo.value(model.Y[i,t])}")
```

**Explanation:**

1. **Import Libraries:** Import necessary libraries like `pyomo` for optimization modeling and `pandas` for data manipulation.
2. **Sample Data:** Provide sample data for the parameters. Replace this with your actual data. I've used dictionary structures here for simplicity. You can read from `.csv` files using `pandas`.
3. **Create Model:** Initialize a concrete model using `pyo.ConcreteModel()`.
4. **Sets:** Define sets for time periods, food products, and beverage products using `pyo.Set()`.
5. **Parameters:** Define parameters like fixed cost, variable cost, demand, and revenue using `pyo.Param()`. You will need to load your actual data into these parameters.
6. **Decision Variables:** Define decision variables for production, storage, and sales using `pyo.Var()`. The `within` argument specifies the domain of the variables. 
7. **Objective Function:** Define the objective function to maximize profit using `pyo.Objective()`. 
8. **Constraints:** Define all the constraints of the problem using `pyo.Constraint()`. 
9. **Solver:** Choose a solver like `cbc` and solve the model using `solver.solve(model)`.
10. **Results:** Print the solver's status and access the optimal values of the decision variables.

**Note:** You will need to install the `pyomo` package and a solver like `cbc` to run this code. You can install them using `pip install pyomo` and `pip install cbc`.

Remember to replace the sample data with your actual data. This code provides a basic structure for your optimization problem. You can customize it further based on your specific needs. 


## 4. Input problem data and try running the generated code

In [1]:
import pyomo.environ as pyo
import pandas as pd

fixed_cost_production = pd.read_csv("fixed_cost_production.csv", usecols=lambda column: column != 0)
variable_cost_production = pd.read_csv("variable_cost_production.csv", usecols=lambda column: column != 0)
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", usecols=lambda column: column != 0)
demand = pd.read_csv("demand.csv", usecols=lambda column: column != 0)
revenue = pd.read_csv("revenue.csv", usecols=lambda column: column != 0)

# Transform into dictionary format
fixed_cost_production_data = {(i, j-1): fixed_cost_production.iloc[i-1, j-1] for i in range(1, len(fixed_cost_production)+1) for j in range(2, len(fixed_cost_production.columns)+1)}
variable_cost_production_data = {(i, j-1): variable_cost_production.iloc[i-1, j-1] for i in range(1, len(variable_cost_production)+1) for j in range(2, len(variable_cost_production.columns)+1)}
variable_cost_storage_data = {(i, j-1): variable_cost_storage.iloc[i-1, j-1] for i in range(1, len(variable_cost_storage)+1) for j in range(2, len(variable_cost_storage.columns)+1)}
demand_data = {(i, j-1): demand.iloc[i-1, j-1] for i in range(1, len(demand)+1) for j in range(2, len(demand.columns)+1)}
revenue_data = {(i, j-1): revenue.iloc[i-1, j-1] for i in range(1, len(revenue)+1) for j in range(2, len(revenue.columns)+1)}


##########

# Sample Data 
# Assuming data is in .csv files, replace with actual file paths if needed
# fixed_cost_production_data = {
#     ('Food1', 1): 100, ('Food1', 2): 105, ('Food1', 3): 110, # ... and so on for all periods and products
#     ('Food2', 1): 150, ('Food2', 2): 155, ('Food2', 3): 160, # ... 
#     ('Beverage1', 1): 80, ('Beverage1', 2): 85, ('Beverage1', 3): 90, # ... 
#     ('Beverage2', 1): 120, ('Beverage2', 2): 125, ('Beverage2', 3): 130, # ... 
#     ('Beverage3', 1): 70, ('Beverage3', 2): 75, ('Beverage3', 3): 80, # ... 
# }
# variable_cost_production_data = {
#     # similar structure as fixed_cost_production_data
# }
# variable_cost_storage_data = {
#     # similar structure as fixed_cost_production_data
# }
# demand_data = {
#     # similar structure as fixed_cost_production_data
# }
# revenue_data = {
#     # similar structure as fixed_cost_production_data
# }

# Create Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, 14))  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[3, 4, 5])  # Beverage products
model.FB = model.F | model.B # All Products

# Parameters
model.FC = pyo.Param(model.FB, model.T, initialize=fixed_cost_production_data) 
model.VCprod = pyo.Param(model.FB, model.T, initialize=variable_cost_production_data)
model.VCstore = pyo.Param(model.FB, model.T, initialize=variable_cost_storage_data)
model.D = pyo.Param(model.FB, model.T, initialize=demand_data)
model.R = pyo.Param(model.FB, model.T, initialize=revenue_data)

# Storage and Production Capacities - you already have this data
model.SC = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
model.PC = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Decision Variables
model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity produced
model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity sold
model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity stored

# Integer requirement for Food Production
model.X_integer = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)
def x_linking_rule(model, i, t):
    if i in model.F:
        return model.X[i,t] == model.X_integer[i,t]
    else:
        return pyo.Constraint.Skip
model.x_linking = pyo.Constraint(model.FB, model.T, rule=x_linking_rule) 

# Objective Function
def objective_rule(model):
    return sum(model.R[i, t] * model.Y[i, t] - model.VCprod[i, t] * model.X[i, t] - model.VCstore[i, t] * model.S[i, t] - model.FC[i, t] * (model.X[i, t] > 0) for i in model.FB for t in model.T)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints

# 1. Production Capacity
def production_capacity_rule(model, i, t):
    return model.X[i, t] <= model.PC[i]
model.production_capacity = pyo.Constraint(model.FB, model.T, rule=production_capacity_rule)

# 2. Storage Capacity
def storage_capacity_rule(model, i, t):
    return model.S[i, t] <= model.SC[i]
model.storage_capacity = pyo.Constraint(model.FB, model.T, rule=storage_capacity_rule)

# 3. Demand
def demand_rule(model, i, t):
    return model.Y[i, t] <= model.D[i, t]
model.demand = pyo.Constraint(model.FB, model.T, rule=demand_rule)

# 4. Inventory Balance
def inventory_balance_rule(model, i, t):
    if t == 1:
        return model.S[i, t] == model.X[i, t] - model.Y[i, t]
    else:
        return model.S[i, t] == model.S[i, t-1] + model.X[i, t] - model.Y[i, t]
model.inventory_balance = pyo.Constraint(model.FB, model.T, rule=inventory_balance_rule)

# Solve the model
solver = pyo.SolverFactory('cbc') # Choose a solver, 'cbc' is a free open-source solver
results = solver.solve(model)

# Print Results
print(results)

# Access the optimal values of decision variables
for i in model.FB:
    for t in model.T:
        print(f"Product {i}, Time {t}: Production = {pyo.value(model.X[i,t])}, Storage = {pyo.value(model.S[i,t])}, Sales = {pyo.value(model.Y[i,t])}")

ERROR: Rule failed when generating expression for Objective profit with index
None: TypeError: unsupported operand type(s) for *: 'float' and
'InequalityExpression'
ERROR: Constructing component 'profit' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'float' and
        'InequalityExpression'


TypeError: unsupported operand type(s) for *: 'float' and 'InequalityExpression'

## 5. Correct the code to verify model viability (optional)

In [ ]:
import pyomo.environ as pyo
import pandas as pd

fixed_cost_production = pd.read_csv("fixed_cost_production.csv", usecols=lambda column: column != 0)
variable_cost_production = pd.read_csv("variable_cost_production.csv", usecols=lambda column: column != 0)
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", usecols=lambda column: column != 0)
demand = pd.read_csv("demand.csv", usecols=lambda column: column != 0)
revenue = pd.read_csv("revenue.csv", usecols=lambda column: column != 0)

# Transform into dictionary format
fixed_cost_production_data = {(i, j-1): fixed_cost_production.iloc[i-1, j-1] for i in range(1, len(fixed_cost_production)+1) for j in range(2, len(fixed_cost_production.columns)+1)}
variable_cost_production_data = {(i, j-1): variable_cost_production.iloc[i-1, j-1] for i in range(1, len(variable_cost_production)+1) for j in range(2, len(variable_cost_production.columns)+1)}
variable_cost_storage_data = {(i, j-1): variable_cost_storage.iloc[i-1, j-1] for i in range(1, len(variable_cost_storage)+1) for j in range(2, len(variable_cost_storage.columns)+1)}
demand_data = {(i, j-1): demand.iloc[i-1, j-1] for i in range(1, len(demand)+1) for j in range(2, len(demand.columns)+1)}
revenue_data = {(i, j-1): revenue.iloc[i-1, j-1] for i in range(1, len(revenue)+1) for j in range(2, len(revenue.columns)+1)}


##########

# Sample Data 
# Assuming data is in .csv files, replace with actual file paths if needed
# fixed_cost_production_data = {
#     ('Food1', 1): 100, ('Food1', 2): 105, ('Food1', 3): 110, # ... and so on for all periods and products
#     ('Food2', 1): 150, ('Food2', 2): 155, ('Food2', 3): 160, # ... 
#     ('Beverage1', 1): 80, ('Beverage1', 2): 85, ('Beverage1', 3): 90, # ... 
#     ('Beverage2', 1): 120, ('Beverage2', 2): 125, ('Beverage2', 3): 130, # ... 
#     ('Beverage3', 1): 70, ('Beverage3', 2): 75, ('Beverage3', 3): 80, # ... 
# }
# variable_cost_production_data = {
#     # similar structure as fixed_cost_production_data
# }
# variable_cost_storage_data = {
#     # similar structure as fixed_cost_production_data
# }
# demand_data = {
#     # similar structure as fixed_cost_production_data
# }
# revenue_data = {
#     # similar structure as fixed_cost_production_data
# }

# Create Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, 13))  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[3, 4, 5])  # Beverage products
model.FB = model.F | model.B # All Products

# Parameters
model.FC = pyo.Param(model.FB, model.T, initialize=fixed_cost_production_data) 
model.VCprod = pyo.Param(model.FB, model.T, initialize=variable_cost_production_data)
model.VCstore = pyo.Param(model.FB, model.T, initialize=variable_cost_storage_data)
model.D = pyo.Param(model.FB, model.T, initialize=demand_data)
model.R = pyo.Param(model.FB, model.T, initialize=revenue_data)

# Storage and Production Capacities - you already have this data
model.SC = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
model.PC = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Decision Variables
model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity produced
model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity sold
model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity stored

# Integer requirement for Food Production
model.X_integer = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)
def x_linking_rule(model, i, t):
    if i in model.F:
        return model.X[i,t] == model.X_integer[i,t]
    else:
        return pyo.Constraint.Skip
model.x_linking = pyo.Constraint(model.FB, model.T, rule=x_linking_rule) 

model.Z = pyo.Var(model.FB, model.T, within=pyo.Binary)

# Link Binary Variable to Production Quantity
def link_binary_production_rule(model, i, t):
    return model.Z[i, t] * model.PC[i] >= model.X[i, t]
model.link_binary_production = pyo.Constraint(model.FB, model.T, rule=link_binary_production_rule)

# Link Binary Variable to Nonzero Production
def link_nonzero_production_rule(model, i, t):
    return model.X[i, t] >= model.Z[i, t]
model.link_nonzero_production = pyo.Constraint(model.FB, model.T, rule=link_nonzero_production_rule)


def objective_rule(model):
    return sum(
        (model.R[i, t] * model.Y[i, t] - 
        model.VCprod[i, t] * model.X[i, t] - 
        model.VCstore[i, t] * model.S[i, t] - 
        model.FC[i, t] * model.Z[i, t]) 
        for i in model.FB 
        for t in model.T
    )
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)
# Constraints

# 1. Production Capacity
def production_capacity_rule(model, i, t):
    return model.X[i, t] <= model.PC[i]
model.production_capacity = pyo.Constraint(model.FB, model.T, rule=production_capacity_rule)

# 2. Storage Capacity
def storage_capacity_rule(model, i, t):
    return model.S[i, t] <= model.SC[i]
model.storage_capacity = pyo.Constraint(model.FB, model.T, rule=storage_capacity_rule)

# 3. Demand
def demand_rule(model, i, t):
    return model.Y[i, t] <= model.D[i, t]
model.demand = pyo.Constraint(model.FB, model.T, rule=demand_rule)

# 4. Inventory Balance
def inventory_balance_rule(model, i, t):
    if t == 1:
        return model.S[i, t] == model.X[i, t] - model.Y[i, t]
    else:
        return model.S[i, t] == model.S[i, t-1] + model.X[i, t] - model.Y[i, t]
model.inventory_balance = pyo.Constraint(model.FB, model.T, rule=inventory_balance_rule)

# Solve the model
solver = pyo.SolverFactory('glpk') # Choose a solver, 'cbc' is a free open-source solver
results = solver.solve(model)

# Print Results
print(results)

# Access the optimal values of decision variables
for i in model.FB:
    for t in model.T:
        print(f"Product {i}, Time {t}: Production = {pyo.value(model.X[i,t])}, Storage = {pyo.value(model.S[i,t])}, Sales = {pyo.value(model.Y[i,t])}")

## 6. Printing the outputs as strings, so they can be saved.

In [2]:
print(response.text)

NameError: name 'response' is not defined

In [ ]:
print(response2.text)